In [ ]:
# develop-hamming-distance-permutation.ipynb
#
# Bryan Daniels, Enrico Borriello
# 2023/9/19
#

In [3]:
import attattach as at
import numpy as np

In [ ]:
at.join_transitions()

In [4]:
def label_to_state (label, digits):
    return np.array(list(map(int,list(format(label,'0'+str(digits)+'b')))))

def state_to_label (state):
    return int(''.join(map(str,state)),2)

In [7]:
def hamming_distance(label1,label2,n):
    return np.sum(abs(label_to_state(label1,n) - label_to_state(label2,n)))

In [11]:
hamming_distance(1,3,n)

1

In [13]:
hamming_distance(1,5,n)

1

In [19]:
hamming_distance(21,21,n)

0

In [16]:
label_to_state(21,n)

array([1, 0, 1, 0, 1])

In [24]:
state_to_label([1,0,1,0,0])

20

In [ ]:
def labels_permutation_close(transitions):
    """
    randomly reassign state labels,
    with bias toward states that are closer in
    Hamming distance
    """
    # Generate a random permutation of the labels
    s = len(transitions)
    p = list(range(s))
    random.shuffle(p)
    # Create a new list of edges with the updated labels
    new_transitions = [(p[i], p[j]) for (i, j) in transitions]
    sorted_transitions = sorted(new_transitions, key=lambda x: x[0])
    return sorted_transitions

In [ ]:
def generate_landscape(num_nodes,landscape_structure,close=True):
    """
    Sample landscape structure: [[3,.25],[1,.50],[1,.05],[2,.20]]
    This corresponds to 4 attractors, of lengths 3, 1, 1, and 2,
    with relative basins sizes equal to 25%, 50%, 5%, and 20%
    """
    
    s = 2**num_nodes # total number of states in the attractor landscape

    # Read the structure of the attractor landscape
    lengths = [B[0] for B in landscape_structure]
    rel_sizes = [B[1] for B in landscape_structure]
    sizes = [int(rel_size*s) for rel_size in rel_sizes] # the last one might be wrong
    sizes[-1] = s-(sum(sizes)-sizes[-1]) # this fixes it
    # attractor states in each basin:
    num_att_states = [ landscape_structure[i][0] for i in range(len(landscape_structure)) ]
    
    # CONDITION 1:
    # 'The sum of the relative basin sizes needs to be 1'
    c1 = np.allclose(sum(rel_sizes),1.) 

    # CONDITION 2:
    # All the basins have at least size 1 
    # (For small n and small relative size of a basin, the product might result in zero states)
    c2 = np.prod([sizes[i] > 0 for i in range(len(sizes))])

    # CONDITION3:
    # There are at least as many states as attractor states
    c3 = np.sum(num_att_states) <= s

    # CONDITION4:
    # There are at least as many states as attractor states **in each individual basin**
    c4 = np.prod([ num_att_states[i] <= sizes[i] for i in range(len(landscape_structure)) ])

    # If all conditions are satisfied, proceed:
    if c1*c2*c3*c4:
    
        # generate the individual basins
        t = []
        for i in range(len(landscape_structure)):
            t.append(transitions(lengths[i], sizes[i]))

        # join them with the sequential relabeling 
        all_t = []
        for i in range(len(t)):
            all_t = join_transitions(all_t,t[i])
    
        if close:
            return labels_permutation_close(all_t)
        else:
            return labels_permutation(all_t)

    else:
        if not c1:
            print('ERROR: The sum of the relative basin sizes is not 1.')
        if not c2:
            print('ERROR: At least one basin has size 0.')
            print('       (relative size is too small for your n).')
        if not c3:
            print('ERROR: There are more attractor states than total states.')
        if not c4:
            print('ERROR: There are more attractor states than total states in at least one basin.')
        return None


# 2023/9/26 Enrico's idea: subgraph isomorphism

If we create a directed graph GH that connects all states within a given Hamming distance threshold, and a second directed graph GT with the desired transition structure, then there exists a permutation of the labels of GT such that transitions are always below the Hamming distance threshold if GT is a subgraph of GH.  And if that isomorphism is found, we can use the corresponding permutation for the `labels_permutation_close` function.

In [25]:
import networkx as nx
from networkx.algorithms import isomorphism

In [43]:
landscape_structure = [[1,.25],[1,.50],[1,.05],[1,.20]]
n = 5
ls = at.generate_landscape(n,landscape_structure)

In [32]:
ls

[(0, 16),
 (1, 3),
 (2, 3),
 (3, 3),
 (4, 10),
 (5, 10),
 (6, 3),
 (7, 10),
 (8, 10),
 (9, 16),
 (10, 10),
 (11, 25),
 (12, 16),
 (13, 10),
 (14, 10),
 (15, 3),
 (16, 16),
 (17, 10),
 (18, 16),
 (19, 10),
 (20, 3),
 (21, 16),
 (22, 10),
 (23, 3),
 (24, 10),
 (25, 10),
 (26, 10),
 (27, 9),
 (28, 30),
 (29, 16),
 (30, 11),
 (31, 31)]

In [44]:
def flip_bits(label,bit_indices,n):
    """
    Flip bits of state corresponding to label at indices given by bit_indices
    """
    state = label_to_state(label,n)
    for i in bit_indices:
        if state[i] == 0:
            state[i] = 1
        else:
            state[i] = 0
    return state_to_label(state)

In [45]:
flip_bits(10,[0,1],10)

778

In [46]:
def hamming_network(n,threshold):
    edgelist = []
    for state in range(2**n):
        for h in range(threshold):
            # TO DO: FIX THE FOLLOWING LINES: CURRENTLY JUST INCLUDES STATES AT HAMMING DISTANCE 1
            for i in range(n):
                bit_indices = [i]
                edgelist.append((state,flip_bits(state,bit_indices,n)))
    return nx.DiGraph(edgelist)

In [51]:
hamming_network(5,1).edges()

OutEdgeView([(0, 16), (0, 8), (0, 4), (0, 2), (0, 1), (16, 0), (16, 24), (16, 20), (16, 18), (16, 17), (8, 24), (8, 0), (8, 12), (8, 10), (8, 9), (4, 20), (4, 12), (4, 0), (4, 6), (4, 5), (2, 18), (2, 10), (2, 6), (2, 0), (2, 3), (1, 17), (1, 9), (1, 5), (1, 3), (1, 0), (17, 1), (17, 25), (17, 21), (17, 19), (17, 16), (9, 25), (9, 1), (9, 13), (9, 11), (9, 8), (5, 21), (5, 13), (5, 1), (5, 7), (5, 4), (3, 19), (3, 11), (3, 7), (3, 1), (3, 2), (18, 2), (18, 26), (18, 22), (18, 16), (18, 19), (10, 26), (10, 2), (10, 14), (10, 8), (10, 11), (6, 22), (6, 14), (6, 2), (6, 4), (6, 7), (19, 3), (19, 27), (19, 23), (19, 17), (19, 18), (11, 27), (11, 3), (11, 15), (11, 9), (11, 10), (7, 23), (7, 15), (7, 3), (7, 5), (7, 6), (20, 4), (20, 28), (20, 16), (20, 22), (20, 21), (12, 28), (12, 4), (12, 8), (12, 14), (12, 13), (21, 5), (21, 29), (21, 17), (21, 23), (21, 20), (13, 29), (13, 5), (13, 9), (13, 15), (13, 12), (22, 6), (22, 30), (22, 18), (22, 20), (22, 23), (14, 30), (14, 6), (14, 10), (14

In [50]:
# example of searching for isomorphisms using NetworkX:
# (see https://networkx.org/documentation/stable/reference/algorithms/isomorphism.vf2.html )
hamming_threshold = 1
GT = nx.DiGraph(ls)
GH = hamming_network(n,hamming_threshold)

DiGM = isomorphism.DiGraphMatcher(GH, GT)
if DiGM.subgraph_is_isomorphic():
    print(DiGM.mapping)
else:
    print("no isomorphism found")

no isomorphism found


In [34]:
DiGM.mapping

{28: 0, 30: 1, 11: 2, 25: 3}

In [29]:
nx.DiGraph([(1,2),(2,3)])